# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

**Question 1**  
**Describe your CNN–RNN architecture in detail. With this architecture in mind, how did you select the values of the variables in Task #1? If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.**  

**Answer:**  
- **Encoder (CNN):** We use a pretrained ResNet-18, strip off the final classification layer, and freeze all its convolutional weights. We then add a linear projection from ResNet’s 512-dim feature vector to a lower-dimensional **embed_size** (256).  
- **Decoder (RNN):** A single‐layer LSTM with input‐size = **embed_size** (256) and hidden‐state size = **hidden_size** (512), followed by a linear layer mapping hidden states to vocabulary logits. We prepend the image embedding as the “first word” to the LSTM input sequence.  
- **Hyperparameter choices:**  
  - **batch_size = 32:** Balances GPU memory use against gradient‐estimate stability.  
  - **vocab_threshold = 5:** Discards very rare words to keep the vocabulary to a few thousand tokens (≈8 000), reducing overfitting and speeding up training.  
  - **vocab_from_file = False:** We rebuild the vocab from scratch on first run to ensure consistency with chosen threshold.  
  - **embed_size = 256:** Common in image‐captioning literature (e.g. 256–512).  
  - **hidden_size = 512:** Provides the decoder enough capacity to model sentence structure without exploding parameter count.  
  - **num_epochs = 3** and **save_every = 1:** Follows the Udacity rubric recommendation and gives multiple checkpoints for inspection.  
  - **print_every = 100** and **log_file:** Defaults that allow us to monitor training loss and perplexity without overwhelming the notebook.  
- **References:**  
  - Vinyals *et al.*, “Show and Tell: A Neural Image Caption Generator” (ICLR 2015)  
  - Xu *et al.*, “Show, Attend and Tell: Neural Image Caption Generation with Visual Attention” (ICML 2015)  

---

**Question 2**  
**How did you select the transform in `transform_train`? If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?**  

**Answer:**  
We retained the provided pipeline because it matches the preprocessing used for ImageNet‐pretrained ResNet models:  
1. **Resize(256)** → ensures the shorter edge is 256 px.  
2. **RandomCrop(224)** → standard input size for ResNet; also acts as data augmentation.  
3. **RandomHorizontalFlip()** → further augmentation to reduce overfitting.  
4. **ToTensor() + Normalize(mean, std)** → scales pixels to [0, 1] and normalizes with ImageNet statistics (mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)).  

Because our encoder is a ResNet pretrained on ImageNet, applying the same normalizations and input size yields representations that transfer well to captioning.  

---

**Question 3**  
**How did you select the trainable parameters of your architecture? Why do you think this is a good choice?**  

**Answer:**  
We freeze the entire ResNet backbone (all convolutional layers) to leverage its pretrained visual features and avoid overfitting on COCO (given our limited compute). We only train:  
```python
params = list(decoder.parameters()) + list(encoder.embed.parameters())
```  
- **`encoder.embed`** (the linear projection from 512→256)  
- **All `decoder`** weights (embedding layer, LSTM, and output linear layer)  

This focuses learning on adapting the image features to caption‐generation and keeps the bulk of ResNet fixed, which speeds up training and reduces the number of parameters to tune.  

---

**Question 4**  
**How did you select the optimizer used to train your model?**  

**Answer:**  
We chose **Adam** with a learning rate of **1 × 10⁻³**, because:  
- Adam combines momentum and adaptive per‐parameter learning rates, which accelerates convergence in RNN‐based models.  
- It is the de facto standard in many image‐captioning implementations (e.g. [Xu et al., 2015] uses Adam).  
- Its default hyperparameters (β₁=0.9, β₂=0.999) work well out of the box, letting us get reasonable captions within 3 epochs without extensive optimizer tuning.

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/mnt/disks/legacy-jupytergpu-data/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## Reviewer Note:
## "Your generated captions all sounded the same because you trained on only 2 000 samples. 
## Increase to 30 000–35 000 examples, retrain, and show two "good" and two "bad" examples."

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 32          # batch size
vocab_threshold = 5       # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 256          # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 30             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 128
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader (TODO: You can change the subset_size, however a 
# very large value would increase training time considerably)
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file,
                         subset_size=30000,
                         cocoapi_loc='/mnt/disks/legacy-jupytergpu-data')

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

# Initialize a list to store losses
losses = []

accumulation_steps = 4  # Define how many steps to accumulate gradients
optimizer.zero_grad()   # Initialize gradients to zero

for epoch in range(1, num_epochs+1):

    for i_step in range(1, total_step+1):

        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)

        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)

        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        loss = loss / accumulation_steps  # Scale the loss by the accumulation steps

        # Store the loss value (scaled back to original)
        losses.append(loss.item() * accumulation_steps)

        # Backward pass.
        loss.backward()

        if i_step % accumulation_steps == 0 or i_step == total_step:
            # Update the parameters in the optimizer and zero the gradients.
            optimizer.step()
            optimizer.zero_grad()

        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item() * accumulation_steps, np.exp(loss.item() * accumulation_steps))

        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()

        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)

    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

In [ ]:
# Plot the training loss curve
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('CrossEntropy Loss')
plt.title('Training Loss over Time')
plt.grid(True)
plt.show()

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.

# Import validation functions
from data_loader import get_loader_val, generate_captions, save_results

# Create validation data loader
val_loader = get_loader_val(
    transform=transform_train,  # Use the same transform as training
    batch_size=1,               # Process one image at a time
    num_workers=0,
    cocoapi_loc='/mnt/disks/legacy-jupytergpu-data'  # Same as training
)

# Generate captions for validation images
results = generate_captions(
    encoder=encoder,
    decoder=decoder,
    data_loader=val_loader,
    device=device,
    vocab=data_loader.dataset.vocab
)

# Save results in COCO format
save_results(results, output_file='captions_val2014_results.json')

# Note on BLEU score calculation:
# To calculate BLEU scores, you can use the COCO caption evaluation tools from:
# https://github.com/tylin/coco-caption
#
# The process involves:
# 1. Clone the repository: git clone https://github.com/tylin/coco-caption.git
# 2. Set up the environment as described in the repository
# 3. Run the evaluation script with our generated captions:
#    python coco-caption/eval.py
#
# Alternatively, you can use the pycocoevalcap package:
# pip install pycocoevalcap
#
# Then calculate scores with:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

# Load ground truth
annotations_file = '/mnt/disks/legacy-jupytergpu-data/cocoapi/annotations/captions_val2014.json'
coco_gt = COCO(annotations_file)

# Create coco_eval object by loading results
coco_res = coco_gt.loadRes('captions_val2014_results.json')

# Create evaluation object
coco_eval = COCOEvalCap(coco_gt, coco_res)

# Evaluate
coco_eval.evaluate()

# Print results
print("\nBLEU Scores:")
for metric, score in coco_eval.eval.items():
    print(f"{metric}: {score:.3f}")

# Analyze results
print("\nModel Evaluation:")
if coco_eval.eval['Bleu_1'] > 0.6:
    print("The model performs well on validation data.")
elif coco_eval.eval['Bleu_1'] > 0.4:
    print("The model performs reasonably on validation data.")
else:
    print("The model may be overfitting to training data.")
